In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

from tqdm import tqdm
from openbb_terminal.sdk import openbb
openbb.keys.fred(key=os.getenv('FRED_API_KEY_1'))
openbb.keys.fmp(key=os.getenv('FMP_RSK_KEY_2'))
import sqlite3
import requests
import time

import warnings
warnings.filterwarnings("ignore")

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

---

### 1. Fetch basic profile data on the S&P 500 constituents

Source: Wiki & Alphavantage

In [2]:
# Pull the S&P 500 constituents profile data from Wiki page (public data)
def fetch_sp500_comp(wiki_url: str=os.getenv('WIKI_SP500_URL')):
    """
    Fetch the S&P 500 constituents profile data from Wiki page (public data)
    """
    wiki_table = pd.read_html(wiki_url)
    sp500_comp_profile = wiki_table[0]
    sp500_comp_profile['Symbol'] = sp500_comp_profile['Symbol'].str.replace('\\.', '-', regex=True)
    sp500_comp_profile.rename(columns={"Symbol": "ticker", "Security": "company_name", "CIK": "cik", "GICS Sector": "sector", 
                                    "GICS Sub-Industry": "sub_industry", "Headquarters Location": "hq", "Founded": "founded", 
                                    "Date added": "date_added"}, inplace=True)
    return sp500_comp_profile

sp500_comp_profile_df = fetch_sp500_comp()
sp500_comp_profile_df.head()

,ticker,company_name,sector,sub_industry,hq,date_added,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


The next block was run earlier and relevant business description data was fetched and stored as a csv file "business_desc.csv". So this block of code can be skipped. 

In [3]:
# # Define stock tickers list
# ticker_list = sp500_comp_profile_df['ticker'].to_list()

# def get_bus_desc_data(tickers: list(str)):
#     """
#     Returns a Series of bus descriptions.
#     """
#     # Initialize an empty dict for the ticker : description data
#     bus_desc = {}
#     # Loop over each ticker symbol and get the company description
#     for tkr in tqdm(tickers):
#         # Define the API endpoint with the ticker symbol and API key
#         API_KEY = os.getenv('ALPHA_VANT_API_KEY')
#         endpoint = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={tkr}&apikey={API_KEY}'
#         print(f"Processing ticker: {tkr}")  # Added to show the ticker being processed
            
#         # Make the API request and extract the company description from the response
#         try:
#             key_stats_response = requests.request("GET", endpoint)
#             response_data = key_stats_response.json()

#             if 'Description' in response_data:
#                 bus_desc[tkr] = response_data['Description']
#             else:
#                 print(f"No 'Description' field found in response for {tkr}")
#                 bus_desc[tkr] = None
                
#             # print(response_data)
#             print(f'{bus_desc}')

#             # Wait for 12 seconds before the next API call to stay within the rate limit of 5 calls per minute
#             time.sleep(12)

#         except Exception as e:
#             print(f"Error for {tkr}: {e}")
#             break

#     bus_desc_df = pd.Series(bus_desc)
#     bus_desc_df = bus_desc_df.reset_index(name='business_desc').rename(columns={'index': 'ticker'})
#     bus_desc_df.to_csv('../data/business_desc.csv', index=False)
#     return bus_desc_df

# bus_desc_df = get_bus_desc_data(tickers=ticker_list)
# # merge the bus desc data with the stocks profile data
# sp500_comp_profile_df = sp500_comp_profile_df.merge(bus_desc_df, how='left', on='ticker')
# # basic profile data for all S&P 500 stocks was concatenated and saved in a csv file "stocks_profile.csv".
# sp500_comp_profile_df.to_csv('../data/stocks_profile.csv', index=False)
    

In [4]:
sp500_comp_profile_df = pd.read_csv('../data/stocks_profile.csv')
print(f'Total stocks in S&P 500 profile data DF: {len(sp500_comp_profile_df)}')
sp500_comp_profile_df.head()

Total stocks in S&P 500 profile data DF: 503


,ticker,company_name,sector,sub_industry,hq,date_added,cik,founded,business_desc
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,The 3M Company is an American multinational co...
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,A. O. Smith Corporation is an American manufac...
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,Abbott Laboratories is an American multination...
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),AbbVie is an American publicly traded biopharm...
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,Accenture plc is an Irish-domiciled multinatio...


---

### 2. Get historical price data for the entire stock universe

Source: OpenBB - AlphaVantage

The next code block was executed and the entire stock universe hostorical price data was fetched and stored in csv file format "stocks_hist_price.csv".

In [5]:
# tickers = sp500_comp_profile_df.loc[:, 'ticker'].to_list()

# def get_hist_price_data(tickers: list(str), start_date: str="2010-01-01", end_date: str="2023-03-31"):
#     """
#     Get historical price data from OpenBB python package - AlphaVantage
#     """
#     stock_dict = {}
#     for tkr in tqdm(tickers):
#         stock_data = openbb.stocks.load(tkr, start_date=start_date, end_date=end_date, source='AlphaVantage', 
#                                         verbose=False).reset_index().rename(columns={"date": "date"})
#         if len(stock_data) > 0:
#             # calculate simple returns & log returns
#             stock_data['simple_return'] = stock_data['Adj Close'].pct_change()
#             stock_data['log_return'] = np.log(stock_data['Adj Close']/stock_data['Adj Close'].shift(1))
#         else:
#             print(f'No data found for {tkr}')

#         stock_dict[tkr] = stock_data
#         time.sleep(11) # wait for 11 seconds before the next API call to stay within the rate limit of 5 calls per minute

#     # create a list of DFs of individual stock historical price data
#     dfs = []
#     for ticker, data in stock_dict.items():
#         data['ticker'] = ticker
#         dfs.append(data)

#     # concatenate all DFs to form a single large DF
#     all_stock_hist_price = pd.concat(dfs, ignore_index=False)
#     # all_stock_hist_price = all_stock_hist_price.drop('index', axis=1)
#     all_stock_hist_price['date'] = pd.to_datetime(all_stock_hist_price['date'])
#     # set the order of columns for better readability
#     column_order = ['ticker'] + [col for col in all_stock_hist_price.columns if col != 'ticker']
#     all_stock_hist_price = all_stock_hist_price[column_order]

#     # Check if any stocks were left out
#     tkr_unique = all_stock_hist_price['ticker'].unique()
#     tkr_dropped = [ tkr for tkr in tickers if tkr not in tkr_unique ]
#     print(f'{len(tkr_dropped)} missed out: {tkr_dropped}') # only 1 stock left out ('IR')
#     # For now, let's not re-fetch the price data for dropped tickers, we'll proceed with what we have  

#     # Create a separate DF containing historical price data of all the stocks and store as a csv file
#     all_stock_hist_price.to_csv('../data/stocks_hist_price.csv', index=False)
#     return all_stock_hist_price

# stocks_hist_price_df = get_hist_price_data(tickers=tickers)

In [6]:
stocks_hist_price_df = pd.read_csv('../data/stocks_hist_price.csv')
print(f'Total count of unique stocks: {len(stocks_hist_price_df.ticker.unique())}\n') # in total, historical price data was fetched for 502 stocks 
stocks_hist_price_df.head()

Total count of unique stocks: 502



,ticker,date,Open,High,Low,Close,Adj Close,Volume,Dividend amount,Split coefficient,simple_return,log_return
0,MMM,2010-01-04,83.09,83.45,82.67,83.02,56.630570,3043700.0,0.0,1.0,NaN,NaN
1,MMM,2010-01-05,82.80,83.23,81.70,82.50,56.275861,2847000.0,0.0,1.0,-0.006264,-0.006283
2,MMM,2010-01-06,83.88,84.60,83.51,83.67,57.073955,5268500.0,0.0,1.0,0.014182,0.014082
3,MMM,2010-01-07,83.32,83.76,82.12,83.73,57.114883,4470100.0,0.0,1.0,0.000717,0.000717
4,MMM,2010-01-08,83.69,84.32,83.30,84.32,57.517341,3405800.0,0.0,1.0,0.007046,0.007022


---

### 3. Get historical Financial Ratios data for all stocks in the defined stock universe

Source: Financial Modeling Prep

We have fetched historical annual meansures from 2010 to 2023 for 56 different financial ratios for S&P 500 constituent stocks 

The next code block was executed and the entire stock universe hostorical ratios data was collected and stored in csv file format "stocks_hist_ratios.csv".

In [7]:
# tickers_list = sp500_comp_profile_df.loc[:, 'ticker'].to_list()

# def get_hist_ratios_data(tickers: list(str), start: int=2010):
#     """
#     Get historical financial ratios data from OpenBB python package - Financial Modeling Prep
#     """
#     ratios_dict = {}
#     for ticker in tqdm(tickers):
#         stock_ratios = openbb.stocks.fa.ratios(ticker, 15)
#         if len(stock_ratios) > 0:
#             stock_ratios = stock_ratios.T
#             stock_ratios = stock_ratios.drop(columns='Period').reset_index()
#             stock_ratios['Fiscal Date Ending'] = stock_ratios['Fiscal Date Ending'].astype(int)
#             stock_ratios = stock_ratios[stock_ratios['Fiscal Date Ending'] >= 2010]
#             stock_ratios = stock_ratios.reindex(index=stock_ratios.index[::-1]).reset_index(drop=True)
#             stock_ratios['ticker'] = ticker
#         else:
#             print(f'No data found for {ticker}')
#         ratios_dict[ticker] = stock_ratios

#     # create a list of DFs of individual stock historical ratios data
#     ratio_dfs = []
#     for tkr, data in ratios_dict.items():
#         ratio_dfs.append(data)

#     # concatenate all DFs to form a single large DF
#     all_stock_hist_ratios = pd.concat(ratio_dfs, ignore_index=False)
#     # all_stock_hist_ratios = all_stock_hist_price.drop('index', axis=1)
#     # set the order of columns for better readability
#     column_order = ['ticker'] + [col for col in all_stock_hist_ratios.columns if col != 'ticker']
#     all_stock_hist_ratios = all_stock_hist_ratios[column_order]

#     # Check if any stocks were left out
#     tkr_unique = all_stock_hist_ratios['ticker'].unique()
#     tkr_dropped = [ tkr for tkr in tickers if tkr not in tkr_unique ]
#     print(f'{len(tkr_dropped)} missed out: {tkr_dropped}') # 1 stock missed out ('PEG')
#     # For now, let's not re-fetch the price data for dropped tickers, we'll proceed with what we have  

#     # Create a separate DF containing historical price data of all the stocks and store as a csv file
#     all_stock_hist_ratios.to_csv('../data/stocks_hist_ratios.csv', index=False)
#     return all_stock_hist_ratios

# stocks_hist_ratios_df = get_hist_ratios_data(tickers=tickers_list)

In [8]:
stocks_hist_ratios_df = pd.read_csv('../data/stocks_hist_ratios.csv')
print(f'Total count of unique stocks: {len(stocks_hist_ratios_df.ticker.unique())}\n') # in total, historical ratios data was fetched for 502 stocks 
stocks_hist_ratios_df.head()

Total count of unique stocks: 502



,ticker,Fiscal Date Ending,Current ratio,Quick ratio,Cash ratio,Days of sales outstanding,Days of inventory outstanding,Operating cycle,Days of payables outstanding,Cash conversion cycle,Gross profit margin,Operating profit margin,Pretax profit margin,Net profit margin,Effective tax rate,Return on assets,Return on equity,Return on capital employed,Net income per EBT,Ebt per ebit,Ebit per revenue,Debt ratio,Debt equity ratio,Long term debt to capitalization,Total debt to capitalization,Cash flow to debt ratio,Company equity multiplier,Receivables turnover,Payables turnover,Inventory turnover,Fixed asset turnover,Asset turnover,Operating cash flow per share,Free cash flow per share,Cash per share,Payout ratio,Operating cash flow sales ratio,Free cash flow operating cash flow ratio,Cash flow coverage ratios,Short term coverage ratios,Capital expenditure coverage ratio,Dividend paid and capex coverage ratio,Dividend payout ratio,Price book value ratio,Price to book ratio,Price to sales ratio,Price earnings ratio,Price to free cash flows ratio,Price to operating cash flows ratio,Price cash flow ratio,Price earnings to growth ratio,Price sales ratio,Dividend yield,Enterprise value multiple,Price fair value,Interest coverage
0,MMM,2010,2.006,1.344,0.555,50.748,83.260,134.009,43.860,90.149,0.481,0.222,0.216,0.153,0.277,0.135,0.261,0.246,0.710,0.972,0.222,0.469,0.903,0.211,0.258,0.949,1.925,7.192,8.322,4.384,3.663,0.884,7.250,5.721,6.274,0.367,0.194,0.789,0.949,4.077,-4.742,12.650,0.367,3.984,3.984,2.341,15.277,15.284,12.061,12.061,0.601,2.341,0.024,10.970,3.984,NaN
1,MMM,2011,2.250,1.407,0.408,49.010,79.452,128.462,38.214,90.248,0.470,0.209,0.204,0.145,0.278,0.135,0.278,0.236,0.710,0.976,0.209,0.498,1.022,0.225,0.251,1.023,2.050,7.447,9.551,4.594,3.863,0.937,7.458,5.512,5.194,0.363,0.178,0.739,1.023,7.748,-3.832,30.023,0.363,4.013,4.013,2.090,14.448,15.846,11.711,11.711,2.504,2.090,0.025,10.553,4.013,NaN
2,MMM,2012,2.198,1.406,0.465,51.093,89.289,140.383,53.011,87.372,0.475,0.217,0.212,0.149,0.290,0.131,0.253,0.234,0.700,0.980,0.217,0.467,0.901,0.219,0.255,0.883,1.928,7.144,6.885,4.088,3.569,0.883,7.638,5.499,6.53,0.368,0.177,0.720,0.883,4.885,-3.571,35.099,0.368,3.980,3.980,2.339,15.739,18.329,13.197,13.197,2.721,2.339,0.023,9.436,3.980,NaN
3,MMM,2013,1.698,1.025,0.344,51.455,87.567,139.023,53.166,85.857,0.478,0.216,0.213,0.151,0.281,0.139,0.266,0.256,0.710,0.984,0.216,0.465,0.891,0.198,0.256,0.968,1.917,7.094,6.865,4.168,3.568,0.92,8.531,6.089,4.894,0.371,0.188,0.714,0.968,3.456,-3.494,89.492,0.371,4.989,4.989,2.828,18.742,21.030,15.011,15.011,2.789,2.828,0.020,11.321,4.989,NaN
4,MMM,2014,1.961,1.14,0.316,49.495,82.245,131.740,51.930,79.810,0.483,0.224,0.221,0.156,0.289,0.158,0.378,0.282,0.705,0.985,0.224,0.58,1.383,0.339,0.343,0.969,2.385,7.375,7.029,4.438,3.748,1.018,10.206,7.907,3.886,0.447,0.208,0.775,0.969,62.509,-4.438,9.165,0.447,8.038,8.038,3.311,21.260,20.527,15.902,15.902,1.815,3.311,0.021,12.925,8.038,NaN


---

### Fetch the common tickers list among all three datasets

In [9]:
profile_tickers = sp500_comp_profile_df.ticker.unique().tolist()
print(f'Tickers in profile dataset ({len(profile_tickers)}):\n{profile_tickers}')
print('--'*20)
price_tickers = stocks_hist_price_df.ticker.unique().tolist()
print(f'Tickers in price dataset ({len(price_tickers)}):\n{price_tickers}')
print('--'*20)
ratio_tickers = stocks_hist_ratios_df.ticker.unique().tolist()
print(f'Tickers in ratios dataset ({len(ratio_tickers)}):\n{ratio_tickers}')
print('--'*20)
common_tickers_universe = [tkr for tkr in profile_tickers if tkr in price_tickers and ratio_tickers]
print(f'\nCommon universe of tickers ({len(common_tickers_universe)}):\n{common_tickers_universe}')

Tickers in profile dataset (503):
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST'

---